In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f  
from pyspark.sql import types as t

In [2]:
#Importes para epata de Transformacao de dado mineracao de dados 
from pyspark.ml.feature import StringIndexer, OneHotEncoder


In [3]:
from pyspark.ml.feature import VectorAssembler

In [4]:
#Padronizacao dos dados (Remocao outiliers)
from pyspark.ml.feature import StandardScaler 

In [5]:
#Imports para criacao de de modelos e validacao
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import DecisionTreeClassifier

from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [6]:
from pyspark.ml.classification import RandomForestClassifier

# Biliotecas do panda


In [7]:
#import pyspark.pandas as ps 

In [8]:
#from IPython.display import display
#import matplotlib.pyplot as plt

In [9]:
#import seaborn as sns

In [10]:
spark = (
    SparkSession.builder
    .master('local')
    .appName('labtcc')
    .getOrCreate()
)

# CENARIO 9


In [11]:
df = spark.read.csv('C:/Users/Belilo/Desktop/SparkJupyter/datasets/ctu13/cenario9.csv', header=True,inferSchema=True)

In [12]:
df.show(5)

+----------+---+-----+-----+-----+-----+-----------+----+----+-------+---------+---------+
|     Label|Dir|Sport|Proto|Dport|State|        Dur|sTos|dTos|TotPkts| TotBytes| SrcBytes|
+----------+---+-----+-----+-----+-----+-----------+----+----+-------+---------+---------+
|Background|<->| 6881|  udp| 6881|  CON|1823.088379|   0|   0|      2|      214|      107|
|Background|<->| 6881|  udp| 6881|  CON|2005.431641|   0|   0|      2|      214|      107|
|Background|<->| 6881|  udp| 6881|  CON|1973.646729|   0|   0|      2|      214|      107|
|Background|<->|35248|  udp|16200|  CON|3599.997803|   0|   0| 409227|207547419|182657149|
|Background|<?>|   80|  tcp|59067|RA_PA|2059.387451|   0|   0| 224275|266462578|  3199174|
+----------+---+-----+-----+-----+-----+-----------+----+----+-------+---------+---------+
only showing top 5 rows



In [13]:
#dfpd = df.toPandas()

In [14]:
#dfpd.head()

In [15]:
# BoxPlot
#dfpd.boxplot(column=['sTos','dTos','Dur'])
#plt.show()

**Contar todos dados do datset**


In [16]:
df.count()

2087508

**Verificar os tipos de dados em cada coluna**

In [17]:
df.printSchema()

root
 |-- Label: string (nullable = true)
 |-- Dir: string (nullable = true)
 |-- Sport: string (nullable = true)
 |-- Proto: string (nullable = true)
 |-- Dport: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Dur: double (nullable = true)
 |-- sTos: integer (nullable = true)
 |-- dTos: integer (nullable = true)
 |-- TotPkts: integer (nullable = true)
 |-- TotBytes: long (nullable = true)
 |-- SrcBytes: integer (nullable = true)



**Remonear colunas**

In [18]:
df = df.withColumnRenamed('Proto','protocolo').withColumnRenamed('Sport','PortaOrigem').withColumnRenamed('Dir','Direcao')\
        .withColumnRenamed('Dport','PortaDestino').withColumnRenamed('State','Estado').withColumnRenamed('Dur','Duracao')\
        .withColumnRenamed('Label',"Rotulo")
df.show()

+----------+-------+-----------+---------+------------+-------+-----------+----+----+-------+---------+---------+
|    Rotulo|Direcao|PortaOrigem|protocolo|PortaDestino| Estado|    Duracao|sTos|dTos|TotPkts| TotBytes| SrcBytes|
+----------+-------+-----------+---------+------------+-------+-----------+----+----+-------+---------+---------+
|Background|    <->|       6881|      udp|        6881|    CON|1823.088379|   0|   0|      2|      214|      107|
|Background|    <->|       6881|      udp|        6881|    CON|2005.431641|   0|   0|      2|      214|      107|
|Background|    <->|       6881|      udp|        6881|    CON|1973.646729|   0|   0|      2|      214|      107|
|Background|    <->|      35248|      udp|       16200|    CON|3599.997803|   0|   0| 409227|207547419|182657149|
|Background|    <?>|         80|      tcp|       59067|  RA_PA|2059.387451|   0|   0| 224275|266462578|  3199174|
|Background|    <?>|       1176|      tcp|          80| FPA_FA|  14.998429|   0|   0|   

**Mostrar Nomes das Colunas**

In [19]:
df.columns

['Rotulo',
 'Direcao',
 'PortaOrigem',
 'protocolo',
 'PortaDestino',
 'Estado',
 'Duracao',
 'sTos',
 'dTos',
 'TotPkts',
 'TotBytes',
 'SrcBytes']

**Selecionar Colunas, atribuir ao novo datafreme**

In [20]:
df = df.select(f.col('Duracao'),f.col('protocolo'),f.col('PortaOrigem'),f.col('Direcao'),f.col('PortaDestino'),f.col('Estado')\
         ,f.col('sTos'),f.col('dTos'),f.col('TotPkts'),f.col('TotBytes'),f.col('SrcBytes'),f.col('Rotulo'))

df.show(5)

+-----------+---------+-----------+-------+------------+------+----+----+-------+---------+---------+----------+
|    Duracao|protocolo|PortaOrigem|Direcao|PortaDestino|Estado|sTos|dTos|TotPkts| TotBytes| SrcBytes|    Rotulo|
+-----------+---------+-----------+-------+------------+------+----+----+-------+---------+---------+----------+
|1823.088379|      udp|       6881|    <->|        6881|   CON|   0|   0|      2|      214|      107|Background|
|2005.431641|      udp|       6881|    <->|        6881|   CON|   0|   0|      2|      214|      107|Background|
|1973.646729|      udp|       6881|    <->|        6881|   CON|   0|   0|      2|      214|      107|Background|
|3599.997803|      udp|      35248|    <->|       16200|   CON|   0|   0| 409227|207547419|182657149|Background|
|2059.387451|      tcp|         80|    <?>|       59067| RA_PA|   0|   0| 224275|266462578|  3199174|Background|
+-----------+---------+-----------+-------+------------+------+----+----+-------+---------+-----

**Alterar o tipo de dados**

In [21]:
df.printSchema()

root
 |-- Duracao: double (nullable = true)
 |-- protocolo: string (nullable = true)
 |-- PortaOrigem: string (nullable = true)
 |-- Direcao: string (nullable = true)
 |-- PortaDestino: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- sTos: integer (nullable = true)
 |-- dTos: integer (nullable = true)
 |-- TotPkts: integer (nullable = true)
 |-- TotBytes: long (nullable = true)
 |-- SrcBytes: integer (nullable = true)
 |-- Rotulo: string (nullable = true)



In [22]:
df = df.withColumn('PortaOrigem',f.col('Portaorigem').cast(t.IntegerType()))\
        .withColumn('PortaDestino',f.col('PortaDestino').cast(t.IntegerType()))

df.printSchema()

root
 |-- Duracao: double (nullable = true)
 |-- protocolo: string (nullable = true)
 |-- PortaOrigem: integer (nullable = true)
 |-- Direcao: string (nullable = true)
 |-- PortaDestino: integer (nullable = true)
 |-- Estado: string (nullable = true)
 |-- sTos: integer (nullable = true)
 |-- dTos: integer (nullable = true)
 |-- TotPkts: integer (nullable = true)
 |-- TotBytes: long (nullable = true)
 |-- SrcBytes: integer (nullable = true)
 |-- Rotulo: string (nullable = true)



**Tratando Valores Nulos / dados faltante**

**Verificar Valores Nulos em cada coluna {Nome_coluna : Qtd de Valores nulos}**

In [23]:
for coluna in df.columns:
    print(coluna," : ", df.filter(df[coluna].isNull()).count())

Duracao  :  0
protocolo  :  0
PortaOrigem  :  24470
Direcao  :  0
PortaDestino  :  23862
Estado  :  3
sTos  :  9031
dTos  :  173232
TotPkts  :  0
TotBytes  :  0
SrcBytes  :  0
Rotulo  :  0


**Preencher Valores nulos apartir do calculo da moda (a frequencia dos dados)**

In [24]:
# A impressao dos valores eh so para mostrar qual eh o valor mais frequente (a moda) em cda uma das colunas que posuem dados nulos
print("Nome da coluna : O valor com maior frequencia (Moda)")

for coluna in df.columns:
    if df.filter(df[coluna].isNull()).count() != 0:
        count_mode_val = df.groupBy(coluna).count().filter(f.col(coluna).isNotNull())\
                            .agg(f.max("count")).collect()[0][0]
        mode_val = df.groupBy(coluna).count().filter(f.col(coluna).isNotNull())\
                            .filter(f.col("count") == count_mode_val).select(coluna).collect()[0][0]

        print(coluna," : ", mode_val)
        df=df.na.fill(mode_val,subset=[coluna])

Nome da coluna : O valor com maior frequencia (Moda)
PortaOrigem  :  53
PortaDestino  :  13363
Estado  :  CON
sTos  :  0
dTos  :  0


**Verificando Novamente Valores nulos**

In [25]:
for coluna in df.columns:
    print(coluna," : ", df.filter(df[coluna].isNull()).count())

Duracao  :  0
protocolo  :  0
PortaOrigem  :  0
Direcao  :  0
PortaDestino  :  0
Estado  :  0
sTos  :  0
dTos  :  0
TotPkts  :  0
TotBytes  :  0
SrcBytes  :  0
Rotulo  :  0


**Tratando dados ruidosos** 

In [26]:
df.createOrReplaceTempView("tabela")
spark.sql("SELECT count(*) from tabela").show()

+--------+
|count(1)|
+--------+
| 2087508|
+--------+



In [27]:
spark.sql("SELECT DISTINCT Duracao from tabela").show()

+-----------+
|    Duracao|
+-----------+
|   6.940647|
|2934.411621|
| 148.079163|
|3542.162598|
|2935.303711|
|    3.71E-4|
|  16.357012|
|3599.464844|
|     7.5E-4|
|   0.130722|
| 191.274338|
|   0.336695|
|   2.020986|
|3176.427979|
|   0.327814|
|3049.628174|
|   0.114029|
|  19.934669|
|  18.339931|
|3006.249512|
+-----------+
only showing top 20 rows



In [28]:
spark.sql("SELECT DISTINCT PortaOrigem from tabela").show()

+-----------+
|PortaOrigem|
+-----------+
|      40574|
|       1238|
|      20683|
|      64590|
|      31261|
|      34061|
|      23364|
|      27974|
|      49855|
|      43527|
|      33569|
|      16574|
|       1342|
|      20497|
|      46465|
|      26755|
|      47501|
|      11748|
|      20735|
|      38723|
+-----------+
only showing top 20 rows



In [29]:
spark.sql("SELECT DISTINCT Direcao from tabela").show(truncate=False)

+-------+
|Direcao|
+-------+
|who    |
|<?     |
|?>     |
|<->    |
|<?>    |
|<-     |
|->     |
+-------+



**O Uso da funcao do replace do spark foi necessaria para tratar dados ruidosos na coluna Direcao**
Tratandosse de dados bidirecional, a coluna deveria ter direcao para duplo sentido, sentido esquedo e direito

In [30]:
# Funcao replace do sql

df=spark.sql("SELECT Duracao, protocolo, PortaOrigem, REPLACE(Direcao, '?', '-') as Direcao, PortaDestino, Estado\
            ,sTos, dTos, TotPkts, TotBytes, SrcBytes, Rotulo  from tabela")

In [31]:
df.createOrReplaceTempView("tabela")
spark.sql("SELECT DISTINCT Direcao from tabela").show(truncate=False)

+-------+
|Direcao|
+-------+
|who    |
|<->    |
|<-     |
|->     |
+-------+



In [32]:
df=df.withColumn('Direcao', f.regexp_replace('Direcao','who','<->'))
df.createOrReplaceTempView("tabela")
spark.sql("SELECT DISTINCT Direcao from tabela").show(truncate=False)

+-------+
|Direcao|
+-------+
|<->    |
|<-     |
|->     |
+-------+



**Elimanar linhas repetidas - a funcao distintct() e a dropDuplicates() (quando nao passado nenhum parametro funcianam da mesma maneira, eliminanam linhas repetidas e retornam um novo datafreme)**

In [33]:
#Total de linhas antes da eliminacao de linhas duplicadas 
df.count()

2087508

In [34]:
# Contar linhas duplicadas 

df.groupBy(df.columns).count().where(f.col('count') > 1).select(f.sum('count')).show()

+----------+
|sum(count)|
+----------+
|     60234|
+----------+



In [35]:
# Total de linhas depois de eliminacao de duplicatas
df.dropDuplicates().count()

2040586

In [36]:
df=df.dropDuplicates()

# Continuacao de preprocessamento - Etapa de Transformacao de dados e Mineracao (aplicando algoritmos de machine learning)

Creando indice para para as variaveis categoricas 

In [37]:
#Novo datafreme para criacao de modelos 
dfparamodelos = df

In [38]:
dfparamodelos.printSchema()

root
 |-- Duracao: double (nullable = true)
 |-- protocolo: string (nullable = true)
 |-- PortaOrigem: integer (nullable = true)
 |-- Direcao: string (nullable = true)
 |-- PortaDestino: integer (nullable = true)
 |-- Estado: string (nullable = false)
 |-- sTos: integer (nullable = true)
 |-- dTos: integer (nullable = true)
 |-- TotPkts: integer (nullable = true)
 |-- TotBytes: long (nullable = true)
 |-- SrcBytes: integer (nullable = true)
 |-- Rotulo: string (nullable = true)



In [39]:
# criar o objeto da classe StringIndexer com especificacao da coluna de entrada e saída (transformar dados categoricos em numericos continuos para criacao de modelos)
SI_protocolo = StringIndexer(inputCol='protocolo',outputCol='protocolo_index')
SI_direcao = StringIndexer(inputCol='Direcao',outputCol='Direcao_index')
SI_estado = StringIndexer(inputCol='Estado',outputCol='Estado_index')
SI_rotulo = StringIndexer(inputCol='Rotulo',outputCol='rotulo_index')

In [40]:
# Aplicando a funcao transfoma(df), nos objectos criados 
dfparamodelos = SI_protocolo.fit(dfparamodelos).transform(dfparamodelos)
dfparamodelos = SI_direcao.fit(dfparamodelos).transform(dfparamodelos)
dfparamodelos = SI_estado.fit(dfparamodelos).transform(dfparamodelos)
dfparamodelos = SI_rotulo.fit(dfparamodelos).transform(dfparamodelos)

In [41]:
dfparamodelos.columns

['Duracao',
 'protocolo',
 'PortaOrigem',
 'Direcao',
 'PortaDestino',
 'Estado',
 'sTos',
 'dTos',
 'TotPkts',
 'TotBytes',
 'SrcBytes',
 'Rotulo',
 'protocolo_index',
 'Direcao_index',
 'Estado_index',
 'rotulo_index']

In [42]:
#vector assembler
assembler = VectorAssembler( inputCols=['Duracao','protocolo_index','PortaOrigem'\
                                        ,'Direcao_index','PortaDestino','Estado_index'\
                                        ,'sTos','dTos','TotPkts','TotBytes','SrcBytes']\
                            ,outputCol='features')

In [43]:
dfmodel = assembler.transform(dfparamodelos)
dfmodel.show(3)

+-----------+---------+-----------+-------+------------+------+----+----+-------+--------+--------+----------+---------------+-------------+------------+------------+--------------------+
|    Duracao|protocolo|PortaOrigem|Direcao|PortaDestino|Estado|sTos|dTos|TotPkts|TotBytes|SrcBytes|    Rotulo|protocolo_index|Direcao_index|Estado_index|rotulo_index|            features|
+-----------+---------+-----------+-------+------------+------+----+----+-------+--------+--------+----------+---------------+-------------+------------+------------+--------------------+
|  28.917549|      tcp|      54145|    <->|       56950|RPA_PA|   0|   0|   6021| 5651087|  138137|Background|            1.0|          0.0|        50.0|         0.0|[28.917549,1.0,54...|
|2476.756836|      tcp|      21456|    <->|         443|FPA_PA|   0|   0|    551|   60009|   32677|Background|            1.0|          0.0|        89.0|         0.0|[2476.756836,1.0,...|
|1830.480713|      udp|      12114|    <->|       18720|   C

In [44]:
dfmodel = dfmodel.select(f.col('features'),f.col('rotulo_index')).withColumnRenamed('rotulo_index','label')

In [45]:
dfmodel.show(3,truncate=False)

+--------------------------------------------------------------------------+-----+
|features                                                                  |label|
+--------------------------------------------------------------------------+-----+
|[28.917549,1.0,54145.0,0.0,56950.0,50.0,0.0,0.0,6021.0,5651087.0,138137.0]|0.0  |
|[2476.756836,1.0,21456.0,0.0,443.0,89.0,0.0,0.0,551.0,60009.0,32677.0]    |0.0  |
|(11,[0,2,4,8,9,10],[1830.480713,12114.0,18720.0,6.0,407.0,223.0])         |0.0  |
+--------------------------------------------------------------------------+-----+
only showing top 3 rows



In [46]:
#BoxPlot antes da normalizacao 
#dfmodelpd = dfmodel.toPandas()

In [47]:
#dfmodelpd.head()

In [48]:
#Padronizacao dos dados (Remocao de outiliers)
standerscaler = StandardScaler().setInputCol('features').setOutputCol('scaled_features')
dfmodeldado = standerscaler.fit(dfmodel).transform(dfmodel)

In [49]:
dfmodeldado.show(3)

+--------------------+-----+--------------------+
|            features|label|     scaled_features|
+--------------------+-----+--------------------+
|[28.917549,1.0,54...|  0.0|[0.03418786751235...|
|[2476.756836,1.0,...|  0.0|[2.92815392374630...|
|(11,[0,2,4,8,9,10...|  0.0|(11,[0,2,4,8,9,10...|
+--------------------+-----+--------------------+
only showing top 3 rows



In [50]:
dadomodel = dfmodeldado.select(f.col('scaled_features'),f.col('label'))
dadomodel.show(3,truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|scaled_features                                                                                                                                                           |label|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|[0.03418786751235035,2.247944335054003,4.477133906307787,0.0,2.767571549195467,7.960832896613092,0.0,0.0,0.4716957947502525,1.0375182817627382,0.08290899759849596]       |0.0  |
|[2.9281539237463052,2.247944335054003,1.7741506158230653,0.0,0.02152825630015087,14.170282555971303,0.0,0.0,0.043166315048561554,0.011017426305824021,0.01961253910629341]|0.0  |
|(11,[0,2,4,8,9,10],[2.1640918495532455,1.0016806748732576,0.9097267673562625,4.7005061758869206E-4,7.472

In [51]:
#Dividir os dados para treino e teste (70% treino, 30% teste)
df_treinamento,df_teste = dadomodel.randomSplit([0.7,0.3])

In [52]:
df_treinamento.count()

1428364

In [53]:
df_teste.count()

612222

**Criacao dos modelos**

**Regressao Logistica**

In [54]:
#Create a logistic regression model object

log_reg=LogisticRegression(labelCol='label', featuresCol='scaled_features', maxIter=50)
log_reg_Modelo=log_reg.fit(df_treinamento)



In [55]:
#teste dos modelo
predicao_teste = log_reg_Modelo.transform(df_teste)


In [56]:
predicao_teste.show(3)

+--------------------+-----+--------------------+--------------------+----------+
|     scaled_features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(11,[0,2,4,8,9,10...|  0.0|[6.70566167356518...|[0.98337973773360...|       0.0|
|(11,[0,2,4,8,9,10...|  0.0|[8.12121403550376...|[0.98111771001225...|       0.0|
|(11,[0,2,4,8,9,10...|  0.0|[9.25039390733130...|[0.97909905325546...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 3 rows



In [57]:
predicaorotulo=predicao_teste.select('label','prediction')

In [58]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
acuracia_LR = Evaluator.evaluate(predicaorotulo)

print("Acuracia: ", acuracia_LR)

Acuracia:  0.9435368216104616


In [59]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='precisionByLabel')
precisao_LR = Evaluator.evaluate(predicaorotulo)

print("Precisao: ", precisao_LR)

Precisao:  0.9624817773215425


In [60]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='recallByLabel')
recal_LR = Evaluator.evaluate(predicaorotulo)

print("Recal: ", recal_LR)

Recal:  0.9756389249693341


**Naive Bayes**

In [61]:
NB= NaiveBayes(labelCol='label',featuresCol='scaled_features', smoothing=1.0)
modeloNB = NB.fit(df_treinamento)

In [62]:
predicao_testeNB = modeloNB.transform(df_teste)

In [63]:
predicao_testeNB.show(3)

+--------------------+-----+--------------------+--------------------+----------+
|     scaled_features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(11,[0,2,4,8,9,10...|  0.0|[-1.3922834046431...|[0.97921385933308...|       0.0|
|(11,[0,2,4,8,9,10...|  0.0|[-1.6947095060392...|[0.97987627439836...|       0.0|
|(11,[0,2,4,8,9,10...|  0.0|[-1.9359706852228...|[0.97980290176649...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 3 rows



In [64]:
predicaorotuloNB=predicao_testeNB.select('label','prediction')

In [65]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
acuracia_NB = Evaluator.evaluate(predicaorotuloNB)

print("Acuracia: ", acuracia_NB)

Acuracia:  0.8670906958586918


In [66]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='precisionByLabel')
precisao_NB = Evaluator.evaluate(predicaorotuloNB)

print("Precisao: ", precisao_NB)

Precisao:  0.9240745230078563


In [67]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='recallByLabel')
recal_NB = Evaluator.evaluate(predicaorotuloNB)

print("Recal: ", recal_NB)

Recal:  0.9296301849707432


**Decision tree**

In [68]:
# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="label", featuresCol="scaled_features"\
                            ,impurity='gini', maxDepth=8, maxBins=32)



In [69]:
dtmodelo=dt.fit(df_treinamento)

In [70]:
predicao_testeDT = dtmodelo.transform(df_teste)

In [71]:
predicao_testeDT.show(3)

+--------------------+-----+--------------------+--------------------+----------+
|     scaled_features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(11,[0,2,4,8,9,10...|  0.0|[436170.0,5.0,370...|[0.99155911912940...|       0.0|
|(11,[0,2,4,8,9,10...|  0.0|[436170.0,5.0,370...|[0.99155911912940...|       0.0|
|(11,[0,2,4,8,9,10...|  0.0|[436170.0,5.0,370...|[0.99155911912940...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 3 rows



In [72]:
predicaorotuloDT=predicao_testeDT.select('label','prediction')

In [73]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
acuracia_DT = Evaluator.evaluate(predicaorotuloDT)

print("Acuracia: ", acuracia_DT)

Acuracia:  0.9754370146776823


In [74]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='precisionByLabel')
precisao_DT = Evaluator.evaluate(predicaorotuloDT)

print("Precisao: ", precisao_DT)

Precisao:  0.9777972186189628


In [75]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='recallByLabel')
recal_DT = Evaluator.evaluate(predicaorotuloDT)

print("Recal: ", recal_DT)

Recal:  0.9954367018518753


**Random Forest Classifier**

In [76]:
rfc = RandomForestClassifier(labelCol='label', featuresCol='scaled_features')


In [77]:
modeloRFC = rfc.fit(df_treinamento)

In [78]:
predicao_testeRFC=modeloRFC.transform(df_teste)

In [79]:
predicao_testeRFC.show(3)

+--------------------+-----+--------------------+--------------------+----------+
|     scaled_features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(11,[0,2,4,8,9,10...|  0.0|[19.4052422270422...|[0.97026211135211...|       0.0|
|(11,[0,2,4,8,9,10...|  0.0|[19.6093330165086...|[0.98046665082543...|       0.0|
|(11,[0,2,4,8,9,10...|  0.0|[19.6093330165086...|[0.98046665082543...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 3 rows



In [80]:
predicaorotuloRFC=predicao_testeRFC.select('label','prediction')

In [81]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
acuracia_RFC = Evaluator.evaluate(predicaorotuloRFC)

print("Acuracia: ", acuracia_RFC)

Acuracia:  0.9678155963033017


In [82]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='precisionByLabel')
precisao_RFC = Evaluator.evaluate(predicaorotuloRFC)

print("Precisao: ", precisao_RFC)

Precisao:  0.967110639779781


In [83]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='recallByLabel')
recal_RFC = Evaluator.evaluate(predicaorotuloRFC)

print("Recal: ", recal_RFC)

Recal:  0.9983560564866217


**CENARIO 10**

In [84]:
df = spark.read.csv('C:/Users/Belilo/Desktop/SparkJupyter/datasets/ctu13/cenario10.csv', header=True,inferSchema=True)

In [85]:
df.show(5)

+----------+-----+-----+---+-----+--------+-----------+----+----+-------+--------+--------+
|     Label|Proto|Sport|Dir|Dport|   State|        Dur|sTos|dTos|TotPkts|TotBytes|SrcBytes|
+----------+-----+-----+---+-----+--------+-----------+----+----+-------+--------+--------+
|Background|  udp|60621|<->|63550|     CON| 2752.65625|   0|   0|      3|     435|     290|
|Background|  udp|51413|<->|63550|     CON|1849.315552|   0|   0|      3|     417|     272|
|Background|  udp|63195|<->|63550|     CON|2091.747314|   0|   0|      2|     290|     145|
|Background|  udp|39110|<->|63550|     CON|1535.769409|   0|   0|      2|     290|     145|
|Background|  tcp|33426|<?>|25443|FRPA_FPA|   0.002636|   0|   0|      6|     490|     321|
+----------+-----+-----+---+-----+--------+-----------+----+----+-------+--------+--------+
only showing top 5 rows



**Contar todos dados do datset**


In [86]:
df.count()

1309791

**Verificar os tipos de dados em cada coluna**

In [87]:
df.printSchema()

root
 |-- Label: string (nullable = true)
 |-- Proto: string (nullable = true)
 |-- Sport: string (nullable = true)
 |-- Dir: string (nullable = true)
 |-- Dport: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Dur: double (nullable = true)
 |-- sTos: integer (nullable = true)
 |-- dTos: integer (nullable = true)
 |-- TotPkts: integer (nullable = true)
 |-- TotBytes: integer (nullable = true)
 |-- SrcBytes: integer (nullable = true)



**Remonear colunas**

In [88]:
df = df.withColumnRenamed('Proto','protocolo').withColumnRenamed('Sport','PortaOrigem').withColumnRenamed('Dir','Direcao')\
        .withColumnRenamed('Dport','PortaDestino').withColumnRenamed('State','Estado').withColumnRenamed('Dur','Duracao')\
        .withColumnRenamed('Label',"Rotulo")
df.show()

+----------+---------+-----------+-------+------------+--------+-----------+----+----+-------+--------+--------+
|    Rotulo|protocolo|PortaOrigem|Direcao|PortaDestino|  Estado|    Duracao|sTos|dTos|TotPkts|TotBytes|SrcBytes|
+----------+---------+-----------+-------+------------+--------+-----------+----+----+-------+--------+--------+
|Background|      udp|      60621|    <->|       63550|     CON| 2752.65625|   0|   0|      3|     435|     290|
|Background|      udp|      51413|    <->|       63550|     CON|1849.315552|   0|   0|      3|     417|     272|
|Background|      udp|      63195|    <->|       63550|     CON|2091.747314|   0|   0|      2|     290|     145|
|Background|      udp|      39110|    <->|       63550|     CON|1535.769409|   0|   0|      2|     290|     145|
|Background|      tcp|      33426|    <?>|       25443|FRPA_FPA|   0.002636|   0|   0|      6|     490|     321|
|Background|      udp|      41915|    <->|       43087|     CON|   72.43679|   0|   0|  23849|24

**Mostrar Nomes das Colunas**

In [89]:
df.columns

['Rotulo',
 'protocolo',
 'PortaOrigem',
 'Direcao',
 'PortaDestino',
 'Estado',
 'Duracao',
 'sTos',
 'dTos',
 'TotPkts',
 'TotBytes',
 'SrcBytes']

**Selecionar Colunas, atribuir ao novo datafreme**

In [90]:
df = df.select(f.col('Duracao'),f.col('protocolo'),f.col('PortaOrigem'),f.col('Direcao'),f.col('PortaDestino'),f.col('Estado')\
         ,f.col('sTos'),f.col('dTos'),f.col('TotPkts'),f.col('TotBytes'),f.col('SrcBytes'),f.col('Rotulo'))

df.show(5)

+-----------+---------+-----------+-------+------------+--------+----+----+-------+--------+--------+----------+
|    Duracao|protocolo|PortaOrigem|Direcao|PortaDestino|  Estado|sTos|dTos|TotPkts|TotBytes|SrcBytes|    Rotulo|
+-----------+---------+-----------+-------+------------+--------+----+----+-------+--------+--------+----------+
| 2752.65625|      udp|      60621|    <->|       63550|     CON|   0|   0|      3|     435|     290|Background|
|1849.315552|      udp|      51413|    <->|       63550|     CON|   0|   0|      3|     417|     272|Background|
|2091.747314|      udp|      63195|    <->|       63550|     CON|   0|   0|      2|     290|     145|Background|
|1535.769409|      udp|      39110|    <->|       63550|     CON|   0|   0|      2|     290|     145|Background|
|   0.002636|      tcp|      33426|    <?>|       25443|FRPA_FPA|   0|   0|      6|     490|     321|Background|
+-----------+---------+-----------+-------+------------+--------+----+----+-------+--------+----

**Alterar o tipo de dados**

In [91]:
df.printSchema()

root
 |-- Duracao: double (nullable = true)
 |-- protocolo: string (nullable = true)
 |-- PortaOrigem: string (nullable = true)
 |-- Direcao: string (nullable = true)
 |-- PortaDestino: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- sTos: integer (nullable = true)
 |-- dTos: integer (nullable = true)
 |-- TotPkts: integer (nullable = true)
 |-- TotBytes: integer (nullable = true)
 |-- SrcBytes: integer (nullable = true)
 |-- Rotulo: string (nullable = true)



In [93]:
df = df.withColumn('PortaOrigem',f.col('Portaorigem').cast(t.IntegerType()))\
        .withColumn('PortaDestino',f.col('PortaDestino').cast(t.IntegerType()))

df.printSchema()

root
 |-- Duracao: double (nullable = true)
 |-- protocolo: string (nullable = true)
 |-- PortaOrigem: integer (nullable = true)
 |-- Direcao: string (nullable = true)
 |-- PortaDestino: integer (nullable = true)
 |-- Estado: string (nullable = true)
 |-- sTos: integer (nullable = true)
 |-- dTos: integer (nullable = true)
 |-- TotPkts: integer (nullable = true)
 |-- TotBytes: integer (nullable = true)
 |-- SrcBytes: integer (nullable = true)
 |-- Rotulo: string (nullable = true)



**Tratando Valores Nulos / dados faltante**

**Verificar Valores Nulos em cada coluna {Nome_coluna : Qtd de Valores nulos}**

In [94]:
for coluna in df.columns:
    print(coluna," : ", df.filter(df[coluna].isNull()).count())

Duracao  :  0
protocolo  :  0
PortaOrigem  :  135176
Direcao  :  0
PortaDestino  :  135709
Estado  :  1228
sTos  :  12852
dTos  :  186791
TotPkts  :  0
TotBytes  :  0
SrcBytes  :  0
Rotulo  :  0


**Preencher Valores nulos apartir do calculo da moda (a frequencia dos dados)**

In [95]:
# A impressao dos valores eh so para mostrar qual eh o valor mais frequente (a moda) em cda uma das colunas que posuem dados nulos
print("Nome da coluna : O valor com maior frequencia (Moda)")

for coluna in df.columns:
    if df.filter(df[coluna].isNull()).count() != 0:
        count_mode_val = df.groupBy(coluna).count().filter(f.col(coluna).isNotNull())\
                            .agg(f.max("count")).collect()[0][0]
        mode_val = df.groupBy(coluna).count().filter(f.col(coluna).isNotNull())\
                            .filter(f.col("count") == count_mode_val).select(coluna).collect()[0][0]

        print(coluna," : ", mode_val)
        df=df.na.fill(mode_val,subset=[coluna])

Nome da coluna : O valor com maior frequencia (Moda)
PortaOrigem  :  13363
PortaDestino  :  53
Estado  :  CON
sTos  :  0
dTos  :  0


**Verificando Novamente Valores nulos**

In [96]:
for coluna in df.columns:
    print(coluna," : ", df.filter(df[coluna].isNull()).count())

Duracao  :  0
protocolo  :  0
PortaOrigem  :  0
Direcao  :  0
PortaDestino  :  0
Estado  :  0
sTos  :  0
dTos  :  0
TotPkts  :  0
TotBytes  :  0
SrcBytes  :  0
Rotulo  :  0


**Tratando dados ruidosos** 

In [97]:
df.createOrReplaceTempView("tabela")
spark.sql("SELECT count(*) from tabela").show()

+--------+
|count(1)|
+--------+
| 1309791|
+--------+



In [98]:
spark.sql("SELECT DISTINCT Duracao from tabela").show()

+-----------+
|    Duracao|
+-----------+
|    7.18E-4|
|3017.522949|
| 3552.17749|
| 416.194489|
|     7.5E-4|
|  516.55481|
| 114.511345|
|    0.00195|
|   0.059421|
|3228.565918|
| 244.545197|
| 309.717377|
|    3.71E-4|
|    0.19075|
|   1.511531|
|   0.179585|
|   0.001894|
|  16.983456|
|3376.620605|
|3096.484131|
+-----------+
only showing top 20 rows



In [99]:
spark.sql("SELECT DISTINCT PortaOrigem from tabela").show()

+-----------+
|PortaOrigem|
+-----------+
|       1238|
|      53565|
|      53963|
|       4935|
|      35820|
|      43527|
|      13840|
|      37111|
|      63155|
|       6620|
|      44822|
|      29894|
|      53634|
|      49717|
|      20924|
|      50348|
|       2866|
|      57201|
|      23336|
|      35361|
+-----------+
only showing top 20 rows



In [100]:
spark.sql("SELECT DISTINCT Direcao from tabela").show(truncate=False)

+-------+
|Direcao|
+-------+
|who    |
|?>     |
|<->    |
|<?>    |
|<-     |
|->     |
+-------+



**O Uso da funcao do replace do spark foi necessaria para tratar dados ruidosos na coluna Direcao**
Tratandosse de dados bidirecional, a coluna deveria ter direcao para duplo sentido, sentido esquedo e direito

In [101]:
# Funcao replace do sql

df=spark.sql("SELECT Duracao, protocolo, PortaOrigem, REPLACE(Direcao, '?', '-') as Direcao, PortaDestino, Estado\
            ,sTos, dTos, TotPkts, TotBytes, SrcBytes, Rotulo  from tabela")

In [102]:
df.createOrReplaceTempView("tabela")
spark.sql("SELECT DISTINCT Direcao from tabela").show(truncate=False)

+-------+
|Direcao|
+-------+
|who    |
|<->    |
|<-     |
|->     |
+-------+



In [103]:
df=df.withColumn('Direcao', f.regexp_replace('Direcao','who','<->'))
df.createOrReplaceTempView("tabela")
spark.sql("SELECT DISTINCT Direcao from tabela").show(truncate=False)

+-------+
|Direcao|
+-------+
|<->    |
|<-     |
|->     |
+-------+



**Elimanar linhas repetidas - a funcao distintct() e a dropDuplicates() (quando nao passado nenhum parametro funcianam da mesma maneira, eliminanam linhas repetidas e retornam um novo datafreme)**

In [104]:
#Total de linhas antes da eliminacao de linhas duplicadas 
df.count()

1309791

In [105]:
# Contar linhas duplicadas 

df.groupBy(df.columns).count().where(f.col('count') > 1).select(f.sum('count')).show()

+----------+
|sum(count)|
+----------+
|    125547|
+----------+



In [106]:
# Total de linhas depois de eliminacao de duplicatas
df.dropDuplicates().count()

1189304

In [107]:
df=df.dropDuplicates()

# Continuacao de preprocessamento - Etapa de Transformacao de dados e Mineracao (aplicando algoritmos de machine learning)

Creando indice para para as variaveis categoricas 

In [108]:
#Novo datafreme para criacao de modelos 
dfparamodelos = df

In [109]:
dfparamodelos.printSchema()

root
 |-- Duracao: double (nullable = true)
 |-- protocolo: string (nullable = true)
 |-- PortaOrigem: integer (nullable = true)
 |-- Direcao: string (nullable = true)
 |-- PortaDestino: integer (nullable = true)
 |-- Estado: string (nullable = false)
 |-- sTos: integer (nullable = true)
 |-- dTos: integer (nullable = true)
 |-- TotPkts: integer (nullable = true)
 |-- TotBytes: integer (nullable = true)
 |-- SrcBytes: integer (nullable = true)
 |-- Rotulo: string (nullable = true)



In [110]:
# criar o objeto da classe StringIndexer com especificacao da coluna de entrada e saída (transformar dados categoricos em numericos continuos para criacao de modelos)
SI_protocolo = StringIndexer(inputCol='protocolo',outputCol='protocolo_index')
SI_direcao = StringIndexer(inputCol='Direcao',outputCol='Direcao_index')
SI_estado = StringIndexer(inputCol='Estado',outputCol='Estado_index')
SI_rotulo = StringIndexer(inputCol='Rotulo',outputCol='rotulo_index')

In [111]:
# Aplicando a funcao transfoma(df), nos objectos criados 
dfparamodelos = SI_protocolo.fit(dfparamodelos).transform(dfparamodelos)
dfparamodelos = SI_direcao.fit(dfparamodelos).transform(dfparamodelos)
dfparamodelos = SI_estado.fit(dfparamodelos).transform(dfparamodelos)
dfparamodelos = SI_rotulo.fit(dfparamodelos).transform(dfparamodelos)

In [112]:
dfparamodelos.columns

['Duracao',
 'protocolo',
 'PortaOrigem',
 'Direcao',
 'PortaDestino',
 'Estado',
 'sTos',
 'dTos',
 'TotPkts',
 'TotBytes',
 'SrcBytes',
 'Rotulo',
 'protocolo_index',
 'Direcao_index',
 'Estado_index',
 'rotulo_index']

In [113]:
#vector assembler
assembler = VectorAssembler( inputCols=['Duracao','protocolo_index','PortaOrigem'\
                                        ,'Direcao_index','PortaDestino','Estado_index'\
                                        ,'sTos','dTos','TotPkts','TotBytes','SrcBytes']\
                            ,outputCol='features')

In [114]:
dfmodel = assembler.transform(dfparamodelos)
dfmodel.show(3)

+-----------+---------+-----------+-------+------------+------+----+----+-------+--------+--------+----------+---------------+-------------+------------+------------+--------------------+
|    Duracao|protocolo|PortaOrigem|Direcao|PortaDestino|Estado|sTos|dTos|TotPkts|TotBytes|SrcBytes|    Rotulo|protocolo_index|Direcao_index|Estado_index|rotulo_index|            features|
+-----------+---------+-----------+-------+------------+------+----+----+-------+--------+--------+----------+---------------+-------------+------------+------------+--------------------+
|   0.126376|      udp|      61043|    <->|          53|   CON|   0|   0|      2|     304|      75|Background|            0.0|          0.0|         0.0|         0.0|(11,[0,2,4,8,9,10...|
|2465.262207|      udp|      49814|    <->|       13363|   CON|   0|   0|     30|    1995|    1095|Background|            0.0|          0.0|         0.0|         0.0|(11,[0,2,4,8,9,10...|
|   0.367005|      udp|      13363|    <->|       29342|   C

In [115]:
dfmodel = dfmodel.select(f.col('features'),f.col('rotulo_index')).withColumnRenamed('rotulo_index','label')

In [116]:
dfmodel.show(3,truncate=False)

+--------------------------------------------------------------------+-----+
|features                                                            |label|
+--------------------------------------------------------------------+-----+
|(11,[0,2,4,8,9,10],[0.126376,61043.0,53.0,2.0,304.0,75.0])          |0.0  |
|(11,[0,2,4,8,9,10],[2465.262207,49814.0,13363.0,30.0,1995.0,1095.0])|0.0  |
|(11,[0,2,4,8,9,10],[0.367005,13363.0,29342.0,3.0,2849.0,60.0])      |0.0  |
+--------------------------------------------------------------------+-----+
only showing top 3 rows



In [117]:
#Padronizacao dos dados (Remocao outiliers)
from pyspark.ml.feature import StandardScaler 

In [118]:
#Padronizacao dos dados (Remocao de outiliers)

standerscaler = StandardScaler().setInputCol('features').setOutputCol('scaled_features')
dfmodeldado = standerscaler.fit(dfmodel).transform(dfmodel)

In [119]:
dfmodeldado.show(3)

+--------------------+-----+--------------------+
|            features|label|     scaled_features|
+--------------------+-----+--------------------+
|(11,[0,2,4,8,9,10...|  0.0|(11,[0,2,4,8,9,10...|
|(11,[0,2,4,8,9,10...|  0.0|(11,[0,2,4,8,9,10...|
|(11,[0,2,4,8,9,10...|  0.0|(11,[0,2,4,8,9,10...|
+--------------------+-----+--------------------+
only showing top 3 rows



In [120]:
dadomodel = dfmodeldado.select(f.col('scaled_features'),f.col('label'))
dadomodel.show(3,truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|scaled_features                                                                                                                                 |label|
+------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|(11,[0,2,4,8,9,10],[1.5741424291037878E-4,3.076349394402171,0.003941705495127226,4.31224717065669E-4,7.579836285365563E-5,5.699986857415242E-5])|0.0  |
|(11,[0,2,4,8,9,10],[3.0707364047799786,2.5104478602419564,0.993830387384625,0.006468370755985035,4.974267562271151E-4,8.321980811826253E-4])    |0.0  |
|(11,[0,2,4,8,9,10],[4.571422914107392E-4,0.6734475199023019,2.182217408264586,6.468370755985035E-4,7.103603150331083E-4,4.559989485932194E-5])  |0.0  |
+---------------------------------------------------------------------------------

In [121]:
#Dividir os dados para treino e teste (70% treino, 30% teste)
df_treinamento,df_teste = dadomodel.randomSplit([0.7,0.3])

In [122]:
df_treinamento.count()

832995

In [123]:
df_teste.count()

356309

**Criacao dos modelos**

**Regressao Logistica**

In [124]:
#Create a logistic regression model object

log_reg=LogisticRegression(labelCol='label', featuresCol='scaled_features', maxIter=50)
log_reg_Modelo=log_reg.fit(df_treinamento)



In [125]:
#teste dos modelo
predicao_teste = log_reg_Modelo.transform(df_teste)


In [126]:
predicao_teste.show(3)

+--------------------+-----+--------------------+--------------------+----------+
|     scaled_features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(11,[0,2,4,8,9,10...|  0.0|[4.09022638969780...|[0.98644101637473...|       0.0|
|(11,[0,2,4,8,9,10...|  0.0|[4.08088448212009...|[0.98653266331523...|       0.0|
|(11,[0,2,4,8,9,10...|  0.0|[4.18324118278243...|[0.98546273233407...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 3 rows



In [127]:
predicaorotulo=predicao_teste.select('label','prediction')

In [128]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
acuracia_LR = Evaluator.evaluate(predicaorotulo)

print("Acuracia: ", acuracia_LR)

Acuracia:  0.9767505171073422


In [129]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='precisionByLabel')
precisao_LR = Evaluator.evaluate(predicaorotulo)

print("Precisao: ", precisao_LR)

Precisao:  0.9791317760507998


In [130]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='recallByLabel')
recal_LR = Evaluator.evaluate(predicaorotulo)

print("Recal: ", recal_LR)

Recal:  0.997777063888618


**Naive Bayes**

In [131]:
NB= NaiveBayes(labelCol='label',featuresCol='scaled_features', smoothing=1.0)
modeloNB = NB.fit(df_treinamento)

In [132]:
predicao_testeNB = modeloNB.transform(df_teste)

In [133]:
predicao_testeNB.show(3)

+--------------------+-----+--------------------+--------------------+----------+
|     scaled_features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(11,[0,2,4,8,9,10...|  0.0|[-1.4302049921824...|[0.98298395572165...|       0.0|
|(11,[0,2,4,8,9,10...|  0.0|[-1.4016927945950...|[0.98304840831441...|       0.0|
|(11,[0,2,4,8,9,10...|  0.0|[-1.7139935989226...|[0.98225557457903...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 3 rows



In [134]:
predicaorotuloNB=predicao_testeNB.select('label','prediction')

In [135]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
acuracia_NB = Evaluator.evaluate(predicaorotuloNB)

print("Acuracia: ", acuracia_NB)

Acuracia:  0.9658077679766719


In [136]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='precisionByLabel')
precisao_NB = Evaluator.evaluate(predicaorotuloNB)

print("Precisao: ", precisao_NB)

Precisao:  0.9862462610002457


In [137]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='recallByLabel')
recal_NB = Evaluator.evaluate(predicaorotuloNB)

print("Recal: ", recal_NB)

Recal:  0.9788261750010039


**Decision tree**

In [138]:
# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="label", featuresCol="scaled_features"\
                            ,impurity='gini', maxDepth=8, maxBins=32)



In [139]:
dtmodelo=dt.fit(df_treinamento)

In [140]:
predicao_testeDT = dtmodelo.transform(df_teste)

In [141]:
predicao_testeDT.show(3)

+--------------------+-----+--------------------+--------------------+----------+
|     scaled_features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(11,[0,2,4,8,9,10...|  0.0|[39299.0,2117.0,0.0]|[0.94888448908634...|       0.0|
|(11,[0,2,4,8,9,10...|  0.0|[39299.0,2117.0,0.0]|[0.94888448908634...|       0.0|
|(11,[0,2,4,8,9,10...|  0.0|[170466.0,223.0,0.0]|[0.99869353033880...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 3 rows



In [142]:
predicaorotuloDT=predicao_testeDT.select('label','prediction')

In [143]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
acuracia_DT = Evaluator.evaluate(predicaorotuloDT)

print("Acuracia: ", acuracia_DT)

Acuracia:  0.9889562149707136


In [144]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='precisionByLabel')
precisao_DT = Evaluator.evaluate(predicaorotuloDT)

print("Precisao: ", precisao_DT)

Precisao:  0.9911712487068929


In [145]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='recallByLabel')
recal_DT = Evaluator.evaluate(predicaorotuloDT)

print("Recal: ", recal_DT)

Recal:  0.9975992289997074


**Random Forest Classifier**

In [146]:
rfc = RandomForestClassifier(labelCol='label', featuresCol='scaled_features')


In [147]:
modeloRFC = rfc.fit(df_treinamento)

In [148]:
predicao_testeRFC=modeloRFC.transform(df_teste)

In [149]:
predicao_testeRFC.show(3)

+--------------------+-----+--------------------+--------------------+----------+
|     scaled_features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(11,[0,2,4,8,9,10...|  0.0|[19.6148821720350...|[0.98074410860175...|       0.0|
|(11,[0,2,4,8,9,10...|  0.0|[19.6307835821788...|[0.98153917910894...|       0.0|
|(11,[0,2,4,8,9,10...|  0.0|[19.7663078095324...|[0.98831539047662...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 3 rows



In [150]:
predicaorotuloRFC=predicao_testeRFC.select('label','prediction')

In [151]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
acuracia_RFC = Evaluator.evaluate(predicaorotuloRFC)

print("Acuracia: ", acuracia_RFC)

Acuracia:  0.9865594189313208


In [152]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='precisionByLabel')
precisao_RFC = Evaluator.evaluate(predicaorotuloRFC)

print("Precisao: ", precisao_RFC)

Precisao:  0.9864993732155035


In [153]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='recallByLabel')
recal_RFC = Evaluator.evaluate(predicaorotuloRFC)

print("Recal: ", recal_RFC)

Recal:  0.9999483705161227


**CENARIO 11**

In [154]:
df = spark.read.csv('C:/Users/Belilo/Desktop/SparkJupyter/datasets/ctu13/cenario11.csv', header=True,inferSchema=True)

In [155]:
df.show(5)

+----------+-----+-----+---+-----+-------+----------+----+----+-------+---------+--------+
|     Label|Proto|Sport|Dir|Dport|  State|       Dur|sTos|dTos|TotPkts| TotBytes|SrcBytes|
+----------+-----+-----+---+-----+-------+----------+----+----+-------+---------+--------+
|Background|  tcp| 1078|<?>|   80|RPA_FPA| 83.062141|   0|   0|  43065| 40974671| 1033777|
|Background|  tcp|13121|<?>|62860| RPA_PA|497.720459|   0|   0| 326962|132430976| 7076046|
|Background|  tcp|   80|<?>| 3088|   PA_A|971.288147|   0|   0|   7912|  7356876| 7153650|
|Background|  tcp|54518|<?>|  993|  PA_PA|899.996399|   0|   0|     48|     5728|    3008|
|Background|  tcp|19083| ?>| 2185|   RPA_| 38.753445|   0|null|      7|      456|     456|
+----------+-----+-----+---+-----+-------+----------+----+----+-------+---------+--------+
only showing top 5 rows



**Contar todos dados do datset**


In [156]:
df.count()

107251

**Verificar os tipos de dados em cada coluna**

In [157]:
df.printSchema()

root
 |-- Label: string (nullable = true)
 |-- Proto: string (nullable = true)
 |-- Sport: string (nullable = true)
 |-- Dir: string (nullable = true)
 |-- Dport: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Dur: double (nullable = true)
 |-- sTos: integer (nullable = true)
 |-- dTos: integer (nullable = true)
 |-- TotPkts: integer (nullable = true)
 |-- TotBytes: integer (nullable = true)
 |-- SrcBytes: integer (nullable = true)



**Remonear colunas**

In [158]:
df = df.withColumnRenamed('Proto','protocolo').withColumnRenamed('Sport','PortaOrigem').withColumnRenamed('Dir','Direcao')\
        .withColumnRenamed('Dport','PortaDestino').withColumnRenamed('State','Estado').withColumnRenamed('Dur','Duracao')\
        .withColumnRenamed('Label',"Rotulo")
df.show()

+----------+---------+-----------+-------+------------+-------+----------+----+----+-------+---------+--------+
|    Rotulo|protocolo|PortaOrigem|Direcao|PortaDestino| Estado|   Duracao|sTos|dTos|TotPkts| TotBytes|SrcBytes|
+----------+---------+-----------+-------+------------+-------+----------+----+----+-------+---------+--------+
|Background|      tcp|       1078|    <?>|          80|RPA_FPA| 83.062141|   0|   0|  43065| 40974671| 1033777|
|Background|      tcp|      13121|    <?>|       62860| RPA_PA|497.720459|   0|   0| 326962|132430976| 7076046|
|Background|      tcp|         80|    <?>|        3088|   PA_A|971.288147|   0|   0|   7912|  7356876| 7153650|
|Background|      tcp|      54518|    <?>|         993|  PA_PA|899.996399|   0|   0|     48|     5728|    3008|
|Background|      tcp|      19083|     ?>|        2185|   RPA_| 38.753445|   0|null|      7|      456|     456|
|Background|      tcp|       3095|    <?>|          80|  RA_PA|662.994385|   0|   0|   6137|  5700818|  

**Mostrar Nomes das Colunas**

In [159]:
df.columns

['Rotulo',
 'protocolo',
 'PortaOrigem',
 'Direcao',
 'PortaDestino',
 'Estado',
 'Duracao',
 'sTos',
 'dTos',
 'TotPkts',
 'TotBytes',
 'SrcBytes']

**Selecionar Colunas, atribuir ao novo datafreme**

In [160]:
df = df.select(f.col('Duracao'),f.col('protocolo'),f.col('PortaOrigem'),f.col('Direcao'),f.col('PortaDestino'),f.col('Estado')\
         ,f.col('sTos'),f.col('dTos'),f.col('TotPkts'),f.col('TotBytes'),f.col('SrcBytes'),f.col('Rotulo'))

df.show(5)

+----------+---------+-----------+-------+------------+-------+----+----+-------+---------+--------+----------+
|   Duracao|protocolo|PortaOrigem|Direcao|PortaDestino| Estado|sTos|dTos|TotPkts| TotBytes|SrcBytes|    Rotulo|
+----------+---------+-----------+-------+------------+-------+----+----+-------+---------+--------+----------+
| 83.062141|      tcp|       1078|    <?>|          80|RPA_FPA|   0|   0|  43065| 40974671| 1033777|Background|
|497.720459|      tcp|      13121|    <?>|       62860| RPA_PA|   0|   0| 326962|132430976| 7076046|Background|
|971.288147|      tcp|         80|    <?>|        3088|   PA_A|   0|   0|   7912|  7356876| 7153650|Background|
|899.996399|      tcp|      54518|    <?>|         993|  PA_PA|   0|   0|     48|     5728|    3008|Background|
| 38.753445|      tcp|      19083|     ?>|        2185|   RPA_|   0|null|      7|      456|     456|Background|
+----------+---------+-----------+-------+------------+-------+----+----+-------+---------+--------+----

**Alterar o tipo de dados**

In [161]:
df.printSchema()

root
 |-- Duracao: double (nullable = true)
 |-- protocolo: string (nullable = true)
 |-- PortaOrigem: string (nullable = true)
 |-- Direcao: string (nullable = true)
 |-- PortaDestino: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- sTos: integer (nullable = true)
 |-- dTos: integer (nullable = true)
 |-- TotPkts: integer (nullable = true)
 |-- TotBytes: integer (nullable = true)
 |-- SrcBytes: integer (nullable = true)
 |-- Rotulo: string (nullable = true)



In [162]:
df = df.withColumn('PortaOrigem',f.col('Portaorigem').cast(t.IntegerType()))\
        .withColumn('PortaDestino',f.col('PortaDestino').cast(t.IntegerType()))

df.printSchema()

root
 |-- Duracao: double (nullable = true)
 |-- protocolo: string (nullable = true)
 |-- PortaOrigem: integer (nullable = true)
 |-- Direcao: string (nullable = true)
 |-- PortaDestino: integer (nullable = true)
 |-- Estado: string (nullable = true)
 |-- sTos: integer (nullable = true)
 |-- dTos: integer (nullable = true)
 |-- TotPkts: integer (nullable = true)
 |-- TotBytes: integer (nullable = true)
 |-- SrcBytes: integer (nullable = true)
 |-- Rotulo: string (nullable = true)



**Tratando Valores Nulos / dados faltante**

**Verificar Valores Nulos em cada coluna {Nome_coluna : Qtd de Valores nulos}**

In [163]:
for coluna in df.columns:
    print(coluna," : ", df.filter(df[coluna].isNull()).count())

Duracao  :  0
protocolo  :  0
PortaOrigem  :  12946
Direcao  :  0
PortaDestino  :  12990
Estado  :  91
sTos  :  980
dTos  :  16959
TotPkts  :  0
TotBytes  :  0
SrcBytes  :  0
Rotulo  :  0


**Preencher Valores nulos apartir do calculo da moda (a frequencia dos dados)**

In [164]:
# A impressao dos valores eh so para mostrar qual eh o valor mais frequente (a moda) em cda uma das colunas que posuem dados nulos
print("Nome da coluna : O valor com maior frequencia (Moda)")

for coluna in df.columns:
    if df.filter(df[coluna].isNull()).count() != 0:
        count_mode_val = df.groupBy(coluna).count().filter(f.col(coluna).isNotNull())\
                            .agg(f.max("count")).collect()[0][0]
        mode_val = df.groupBy(coluna).count().filter(f.col(coluna).isNotNull())\
                            .filter(f.col("count") == count_mode_val).select(coluna).collect()[0][0]

        print(coluna," : ", mode_val)
        df=df.na.fill(mode_val,subset=[coluna])

Nome da coluna : O valor com maior frequencia (Moda)
PortaOrigem  :  13363
PortaDestino  :  53
Estado  :  CON
sTos  :  0
dTos  :  0


**Verificando Novamente Valores nulos**

In [165]:
for coluna in df.columns:
    print(coluna," : ", df.filter(df[coluna].isNull()).count())

Duracao  :  0
protocolo  :  0
PortaOrigem  :  0
Direcao  :  0
PortaDestino  :  0
Estado  :  0
sTos  :  0
dTos  :  0
TotPkts  :  0
TotBytes  :  0
SrcBytes  :  0
Rotulo  :  0


**Tratando dados ruidosos** 

In [166]:
df.createOrReplaceTempView("tabela")
spark.sql("SELECT count(*) from tabela").show()

+--------+
|count(1)|
+--------+
|  107251|
+--------+



In [167]:
spark.sql("SELECT DISTINCT Duracao from tabela").show()

+----------+
|   Duracao|
+----------+
|252.235641|
| 965.60437|
|948.458923|
|   0.38899|
|950.313721|
|932.998535|
| 29.553505|
|    7.5E-4|
|   3.71E-4|
| 33.070011|
| 21.092537|
|   7.18E-4|
|  0.154451|
|  1.244861|
|   8.77E-4|
|  0.999027|
|  0.218348|
|  0.187115|
|862.619385|
|  0.028589|
+----------+
only showing top 20 rows



In [168]:
spark.sql("SELECT DISTINCT PortaOrigem from tabela").show()

+-----------+
|PortaOrigem|
+-----------+
|      65408|
|      15790|
|      43688|
|       4818|
|      43527|
|      52697|
|       4935|
|      49331|
|      40386|
|      51415|
|       2122|
|      30970|
|      57693|
|      17420|
|      53634|
|      49308|
|       1088|
|      50353|
|      12799|
|      56110|
+-----------+
only showing top 20 rows



In [169]:
spark.sql("SELECT DISTINCT Direcao from tabela").show(truncate=False)

+-------+
|Direcao|
+-------+
|who    |
|?>     |
|<->    |
|<?>    |
|<-     |
|->     |
+-------+



**O Uso da funcao do replace do spark foi necessaria para tratar dados ruidosos na coluna Direcao**
Tratandosse de dados bidirecional, a coluna deveria ter direcao para duplo sentido, sentido esquedo e direito

In [170]:
# Funcao replace do sql

df=spark.sql("SELECT Duracao, protocolo, PortaOrigem, REPLACE(Direcao, '?', '-') as Direcao, PortaDestino, Estado\
            ,sTos, dTos, TotPkts, TotBytes, SrcBytes, Rotulo  from tabela")

In [171]:
df.createOrReplaceTempView("tabela")
spark.sql("SELECT DISTINCT Direcao from tabela").show(truncate=False)

+-------+
|Direcao|
+-------+
|who    |
|<->    |
|<-     |
|->     |
+-------+



In [172]:
df=df.withColumn('Direcao', f.regexp_replace('Direcao','who','<->'))
df.createOrReplaceTempView("tabela")
spark.sql("SELECT DISTINCT Direcao from tabela").show(truncate=False)

+-------+
|Direcao|
+-------+
|<->    |
|<-     |
|->     |
+-------+



**Elimanar linhas repetidas - a funcao distintct() e a dropDuplicates() (quando nao passado nenhum parametro funcianam da mesma maneira, eliminanam linhas repetidas e retornam um novo datafreme)**

In [173]:
#Total de linhas antes da eliminacao de linhas duplicadas 
df.count()

107251

In [174]:
# Contar linhas duplicadas 

df.groupBy(df.columns).count().where(f.col('count') > 1).select(f.sum('count')).show()

+----------+
|sum(count)|
+----------+
|     10675|
+----------+



In [175]:
# Total de linhas depois de eliminacao de duplicatas
df.dropDuplicates().count()

97178

In [176]:
df=df.dropDuplicates()

# Continuacao de preprocessamento - Etapa de Transformacao de dados e Mineracao (aplicando algoritmos de machine learning)

Creando indice para para as variaveis categoricas 

In [177]:
#Novo datafreme para criacao de modelos 
dfparamodelos = df

In [178]:
dfparamodelos.printSchema()

root
 |-- Duracao: double (nullable = true)
 |-- protocolo: string (nullable = true)
 |-- PortaOrigem: integer (nullable = true)
 |-- Direcao: string (nullable = true)
 |-- PortaDestino: integer (nullable = true)
 |-- Estado: string (nullable = false)
 |-- sTos: integer (nullable = true)
 |-- dTos: integer (nullable = true)
 |-- TotPkts: integer (nullable = true)
 |-- TotBytes: integer (nullable = true)
 |-- SrcBytes: integer (nullable = true)
 |-- Rotulo: string (nullable = true)



In [179]:
# criar o objeto da classe StringIndexer com especificacao da coluna de entrada e saída (transformar dados categoricos em numericos continuos para criacao de modelos)
SI_protocolo = StringIndexer(inputCol='protocolo',outputCol='protocolo_index')
SI_direcao = StringIndexer(inputCol='Direcao',outputCol='Direcao_index')
SI_estado = StringIndexer(inputCol='Estado',outputCol='Estado_index')
SI_rotulo = StringIndexer(inputCol='Rotulo',outputCol='rotulo_index')

In [180]:
# Aplicando a funcao transfoma(df), nos objectos criados 
dfparamodelos = SI_protocolo.fit(dfparamodelos).transform(dfparamodelos)
dfparamodelos = SI_direcao.fit(dfparamodelos).transform(dfparamodelos)
dfparamodelos = SI_estado.fit(dfparamodelos).transform(dfparamodelos)
dfparamodelos = SI_rotulo.fit(dfparamodelos).transform(dfparamodelos)

In [181]:
dfparamodelos.columns

['Duracao',
 'protocolo',
 'PortaOrigem',
 'Direcao',
 'PortaDestino',
 'Estado',
 'sTos',
 'dTos',
 'TotPkts',
 'TotBytes',
 'SrcBytes',
 'Rotulo',
 'protocolo_index',
 'Direcao_index',
 'Estado_index',
 'rotulo_index']

In [182]:
#vector assembler
assembler = VectorAssembler( inputCols=['Duracao','protocolo_index','PortaOrigem'\
                                        ,'Direcao_index','PortaDestino','Estado_index'\
                                        ,'sTos','dTos','TotPkts','TotBytes','SrcBytes']\
                            ,outputCol='features')

In [183]:
dfmodel = assembler.transform(dfparamodelos)
dfmodel.show(3)

+---------+---------+-----------+-------+------------+---------+----+----+-------+--------+--------+----------+---------------+-------------+------------+------------+--------------------+
|  Duracao|protocolo|PortaOrigem|Direcao|PortaDestino|   Estado|sTos|dTos|TotPkts|TotBytes|SrcBytes|    Rotulo|protocolo_index|Direcao_index|Estado_index|rotulo_index|            features|
+---------+---------+-----------+-------+------------+---------+----+----+-------+--------+--------+----------+---------------+-------------+------------+------------+--------------------+
|  8.11E-4|      udp|      45599|    <->|       13363|      CON|   0|   0|      2|     312|      64|Background|            0.0|          0.0|         0.0|         0.0|(11,[0,2,4,8,9,10...|
|28.561224|      tcp|      38572|     ->|         443|SRPA_FSPA|   0|   0|     26|    8728|    6404|Background|            1.0|          1.0|         3.0|         0.0|[28.561224,1.0,38...|
|  2.32E-4|      udp|      57052|    <->|          53| 

In [184]:
dfmodel = dfmodel.select(f.col('features'),f.col('rotulo_index')).withColumnRenamed('rotulo_index','label')

In [185]:
dfmodel.show(3,truncate=False)

+----------------------------------------------------------------+-----+
|features                                                        |label|
+----------------------------------------------------------------+-----+
|(11,[0,2,4,8,9,10],[8.11E-4,45599.0,13363.0,2.0,312.0,64.0])    |0.0  |
|[28.561224,1.0,38572.0,1.0,443.0,3.0,0.0,0.0,26.0,8728.0,6404.0]|0.0  |
|(11,[0,2,4,8,9,10],[2.32E-4,57052.0,53.0,2.0,208.0,79.0])       |0.0  |
+----------------------------------------------------------------+-----+
only showing top 3 rows



In [186]:
#Padronizacao dos dados (Remocao outiliers)
from pyspark.ml.feature import StandardScaler 

In [187]:
#Padronizacao dos dados (Remocao de outiliers)

standerscaler = StandardScaler().setInputCol('features').setOutputCol('scaled_features')
dfmodeldado = standerscaler.fit(dfmodel).transform(dfmodel)

In [188]:
dfmodeldado.show(3)

+--------------------+-----+--------------------+
|            features|label|     scaled_features|
+--------------------+-----+--------------------+
|(11,[0,2,4,8,9,10...|  0.0|(11,[0,2,4,8,9,10...|
|[28.561224,1.0,38...|  0.0|[0.16159247410844...|
|(11,[0,2,4,8,9,10...|  0.0|(11,[0,2,4,8,9,10...|
+--------------------+-----+--------------------+
only showing top 3 rows



In [189]:
dadomodel = dfmodeldado.select(f.col('scaled_features'),f.col('label'))
dadomodel.show(3,truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|scaled_features                                                                                                                                                                           |label|
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|(11,[0,2,4,8,9,10],[4.588441185221788E-6,2.3410945494792372,0.8962376031382945,6.779452870716476E-4,1.122400961373728E-4,4.051831767886951E-4])                                           |0.0  |
|[0.16159247410844013,1.8808911201130725,1.9803219141321768,2.219103005214615,0.02971138652924227,0.3477020085615872,0.0,0.0,0.008813288731931419,0.0031398447406634287,0.0405436416274188]|0.0  |
|(11,[0,2,4,8,9,10],[1.31

In [190]:
#Dividir os dados para treino e teste (70% treino, 30% teste)
df_treinamento,df_teste = dadomodel.randomSplit([0.7,0.3])

In [191]:
df_treinamento.count()

68307

In [192]:
df_teste.count()

28871

**Criacao dos modelos**

**Regressao Logistica**

In [193]:
#Create a logistic regression model object

log_reg=LogisticRegression(labelCol='label', featuresCol='scaled_features', maxIter=50)
log_reg_Modelo=log_reg.fit(df_treinamento)



In [194]:
#teste dos modelo
predicao_teste = log_reg_Modelo.transform(df_teste)


In [195]:
predicao_teste.show(3)

+--------------------+-----+--------------------+--------------------+----------+
|     scaled_features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(11,[0,2,4,8,9,10...|  0.0|[19.2227210977684...|[0.99999999999922...|       0.0|
|(11,[0,2,4,8,9,10...|  0.0|[26.0249648785573...|[1.0,2.8973526758...|       0.0|
|(11,[0,2,4,8,9,10...|  0.0|[4.08851001618415...|[0.98683411671268...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 3 rows



In [196]:
predicaorotulo=predicao_teste.select('label','prediction')

In [197]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
acuracia_LR = Evaluator.evaluate(predicaorotulo)

print("Acuracia: ", acuracia_LR)

Acuracia:  0.9850022513941326


In [198]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='precisionByLabel')
precisao_LR = Evaluator.evaluate(predicaorotulo)

print("Precisao: ", precisao_LR)

Precisao:  0.9865745933819405


In [199]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='recallByLabel')
recal_LR = Evaluator.evaluate(predicaorotulo)

print("Recal: ", recal_LR)

Recal:  0.9982620415691282


**Naive Bayes**

In [200]:
NB= NaiveBayes(labelCol='label',featuresCol='scaled_features', smoothing=1.0)
modeloNB = NB.fit(df_treinamento)

In [201]:
predicao_testeNB = modeloNB.transform(df_teste)

In [202]:
predicao_testeNB.show(3)

+--------------------+-----+--------------------+--------------------+----------+
|     scaled_features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(11,[0,2,4,8,9,10...|  0.0|[-3.2758189835748...|[0.99993487892554...|       0.0|
|(11,[0,2,4,8,9,10...|  0.0|[-4.6446100824540...|[0.99999404864930...|       0.0|
|(11,[0,2,4,8,9,10...|  0.0|[-1.6992237042355...|[0.99524008100203...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 3 rows



In [203]:
predicaorotuloNB=predicao_testeNB.select('label','prediction')

In [204]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
acuracia_NB = Evaluator.evaluate(predicaorotuloNB)

print("Acuracia: ", acuracia_NB)

Acuracia:  0.9606871947629109


In [205]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='precisionByLabel')
precisao_NB = Evaluator.evaluate(predicaorotuloNB)

print("Precisao: ", precisao_NB)

Precisao:  0.9939246216530849


In [206]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='recallByLabel')
recal_NB = Evaluator.evaluate(predicaorotuloNB)

print("Recal: ", recal_NB)

Recal:  0.9690359650989572


**Decision tree**

In [207]:
# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="label", featuresCol="scaled_features"\
                            ,impurity='gini', maxDepth=8, maxBins=32)



In [208]:
dtmodelo=dt.fit(df_treinamento)

In [209]:
predicao_testeDT = dtmodelo.transform(df_teste)

In [210]:
predicao_testeDT.show(3)

+--------------------+-----+------------------+--------------------+----------+
|     scaled_features|label|     rawPrediction|         probability|prediction|
+--------------------+-----+------------------+--------------------+----------+
|(11,[0,2,4,8,9,10...|  0.0| [29648.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|(11,[0,2,4,8,9,10...|  0.0| [29648.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|(11,[0,2,4,8,9,10...|  0.0|[16542.0,42.0,0.0]|[0.99746743849493...|       0.0|
+--------------------+-----+------------------+--------------------+----------+
only showing top 3 rows



In [211]:
predicaorotuloDT=predicao_testeDT.select('label','prediction')

In [212]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
acuracia_DT = Evaluator.evaluate(predicaorotuloDT)

print("Acuracia: ", acuracia_DT)

Acuracia:  0.9939039174257905


In [213]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='precisionByLabel')
precisao_DT = Evaluator.evaluate(predicaorotuloDT)

print("Precisao: ", precisao_DT)

Precisao:  0.9965969514356611


In [214]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='recallByLabel')
recal_DT = Evaluator.evaluate(predicaorotuloDT)

print("Recal: ", recal_DT)

Recal:  0.9971625168475562


**Random Forest Classifier**

In [215]:
rfc = RandomForestClassifier(labelCol='label', featuresCol='scaled_features')


In [216]:
modeloRFC = rfc.fit(df_treinamento)

In [217]:
predicao_testeRFC=modeloRFC.transform(df_teste)

In [218]:
predicao_testeRFC.show(3)

+--------------------+-----+--------------------+--------------------+----------+
|     scaled_features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(11,[0,2,4,8,9,10...|  0.0|[19.8671305212248...|[0.99335652606124...|       0.0|
|(11,[0,2,4,8,9,10...|  0.0|[19.8671305212248...|[0.99335652606124...|       0.0|
|(11,[0,2,4,8,9,10...|  0.0|[19.8503673081434...|[0.99251836540717...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 3 rows



In [219]:
predicaorotuloRFC=predicao_testeRFC.select('label','prediction')

In [220]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
acuracia_RFC = Evaluator.evaluate(predicaorotuloRFC)

print("Acuracia: ", acuracia_RFC)

Acuracia:  0.9938692805929826


In [221]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='precisionByLabel')
precisao_RFC = Evaluator.evaluate(predicaorotuloRFC)

print("Precisao: ", precisao_RFC)

Precisao:  0.9940050075819021


In [222]:
Evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='recallByLabel')
recal_RFC = Evaluator.evaluate(predicaorotuloRFC)

print("Recal: ", recal_RFC)

Recal:  0.9997517202241611
